In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2

In [ ]:
from superglue.keypoint_extractors import SuperPointExtractor, Keypoints
from superglue.datasets import create_training_image_pair_sampler_fn

In [ ]:
extractor = SuperPointExtractor("/home/mkk/Transferências/sp_v5")

In [ ]:
image_size=(512, 512)

sample_fn = create_training_image_pair_sampler_fn(
    extractor=extractor,
    image_size=image_size,
    reprojection_threshold = 5, 
    num_matches=512,
)

In [ ]:
image = Image.open("/home/mkk/Downloads/a.jpg")
image_np = np.array(image)

In [ ]:
input_image = np.array(image.resize(image_size))
transformed_image, (features, labels) = sample_fn(image_np)

In [ ]:
fa, ka, fb, kb = features

In [ ]:
cv_ka = Keypoints.convert_numpy_keypoints_to_cv(ka.numpy() * np.array([[512, 512]]))
cv_kb = Keypoints.convert_numpy_keypoints_to_cv(kb.numpy() * np.array([[512, 512]]))

In [ ]:
np.where(np.diff(labels[:, -1]) == 1)[0]

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    input_image, 
    cv_ka, 
    outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    transformed_image.numpy().astype("uint8"), 
    cv_kb, 
    outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
np.where(np.diff(labels[:, -1]) == 1)[0][0]

In [ ]:
start = 450
num_matches = 512
img_with_keypoints = cv2.drawMatches(
    input_image, cv_ka[start:num_matches],
    transformed_image.numpy().astype("uint8"), cv_kb[start:num_matches],
    [cv2.DMatch(_queryIdx=i, _trainIdx=i, _distance=0) for i in range(num_matches - start)], outImg=None
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
from superglue.homographies import homographic_augmentation, invert_homography, warp_points

In [ ]:
data = {"image": image_np, "keypoints": keypoints.keypoints}
transformed_data = homographic_augmentation(data, add_homography=True, params={}, valid_border_margin=0)

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    transformed_data['image'].numpy(), 
    Keypoints.convert_numpy_keypoints_to_cv(transformed_data['keypoints'].numpy()), 
    outImage=None, color=(0, 0, 255), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
Image.fromarray(img_with_keypoints)

In [ ]:
# Image.fromarray(transformed_data['image'].numpy())
keypoints2 = extractor.extract(transformed_data['image'])

In [ ]:
len(keypoints2.keypoints)

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    transformed_data['image'].numpy(), 
    keypoints2.cv_keypoints, outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
# transformed_data['keypoints']

In [ ]:
import tensorflow as tf

In [ ]:
inv_homography = invert_homography(tf.expand_dims(transformed_data['homography'], 0))

In [ ]:
reprojected_points = warp_points(keypoints2.numpy_keypoints(), inv_homography[0])

In [ ]:
img_with_keypoints = cv2.drawKeypoints(
    image_np, Keypoints.convert_numpy_keypoints_to_cv(reprojected_points.numpy()), outImage=None, color=(0, 0, 255), 
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

In [ ]:
Image.fromarray(img_with_keypoints)

In [ ]:
from scipy.spatial import distance_matrix

In [ ]:
match_matrix = distance_matrix(keypoints.numpy_keypoints(), reprojected_points.numpy())

In [ ]:
k1 = keypoints.numpy_keypoints()
k2 = reprojected_points.numpy()
dmat = np.sqrt(np.maximum(np.sum(k1**2, -1, keepdims=True) - 2 * k1 @ k2.T + np.sum(k2**2, -1, keepdims=True).T, 0.0))
dmat

In [ ]:
np.max(np.abs(match_matrix - dmat))

In [ ]:
dmat.min(-1) < 5 